In [1]:
#import
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LassoCV
from sklearn import metrics
import warnings
warnings.filterwarnings('ignore')

In [2]:
#Reading dataset
df=pd.read_csv('DFT.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 22 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Cate_name          68 non-null     object 
 1   Cate_HOMO          68 non-null     float64
 2   Cate_LUMO          68 non-null     float64
 3   Cate_MC_O1         68 non-null     float64
 4   Cate_MC_H1         68 non-null     float64
 5   Cate_MC_O2         68 non-null     float64
 6   Cate_MC_H2         68 non-null     float64
 7   Cate_BL_ArO1       68 non-null     float64
 8   Cate_BL_O1H1       68 non-null     float64
 9   Cate_BL_ArO2       68 non-null     float64
 10  Cate_BL_O2H2       68 non-null     float64
 11  Cate_total_dipole  68 non-null     float64
 12  P_name             68 non-null     object 
 13  P_HOMO             68 non-null     float64
 14  P_LUMO             68 non-null     float64
 15  P_MC_P             68 non-null     float64
 16  P_MC_=O            68 non-nu

In [3]:
#Building descriptors
X = df.drop(columns=['Yield', 'Cate_name', 'P_name' ])
print('---Descriptors---')
print(X.head())

y = pd.DataFrame(df['Yield'],columns=['Yield'])
print('---Objective---')
print(y.head())

---Descriptors---
   Cate_HOMO  Cate_LUMO  Cate_MC_O1  Cate_MC_H1  Cate_MC_O2  Cate_MC_H2  \
0  -5.940433  -0.165443   -0.374537    0.264452   -0.416200    0.277039   
1  -5.940433  -0.165443   -0.374537    0.264452   -0.416200    0.277039   
2  -5.940433  -0.165443   -0.374537    0.264452   -0.416200    0.277039   
3  -5.940433  -0.165443   -0.374537    0.264452   -0.416200    0.277039   
4  -5.807916  -0.157824   -0.376813    0.263887   -0.418652    0.276114   

   Cate_BL_ArO1  Cate_BL_O1H1  Cate_BL_ArO2  Cate_BL_O2H2  Cate_total_dipole  \
0       1.36318       0.96622       1.37543       0.96253             2.8219   
1       1.36318       0.96622       1.37543       0.96253             2.8219   
2       1.36318       0.96622       1.37543       0.96253             2.8219   
3       1.36318       0.96622       1.37543       0.96253             2.8219   
4       1.36386       0.96625       1.37631       0.96239             2.5961   

     P_HOMO    P_LUMO    P_MC_P   P_MC_=O    P_MC_

In [4]:
#storage 
data_r2_train =[]
data_RMSE_train = []
data_MAE_train = []
data_r2_test = []
data_RMSE_test = []
data_MAE_test = []
cum_coef = np.zeros((1,X.shape[1]))

#Lasso
for i in range(10):
    seed=i
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=seed)
    
    a_X_train = (X_train - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
    a_X_test = (X_test - X_train.mean(axis=0)) / X_train.std(axis=0, ddof=1)
    
    n_lambda = 1000
    alphas = np.logspace(-6, 0, n_lambda)
    model = LassoCV(cv=5, alphas=alphas)
    model.fit(a_X_train, y_train['Yield'])
    
    y_pred1 = model.predict(a_X_train)
    y_pred2 = model.predict(a_X_test)
    
    #train
    r2_train = metrics.r2_score(y_train, y_pred1)
    RMSE_train = metrics.root_mean_squared_error(y_train, y_pred1)
    MAE_train =  metrics.mean_absolute_error(y_train, y_pred1)
    #test
    r2_test = metrics.r2_score(y_test, y_pred2)
    RMSE_test = metrics.root_mean_squared_error(y_test, y_pred2)
    MAE_test = metrics.mean_absolute_error(y_test, y_pred2)
    
    data_r2_train.append(r2_train)
    data_RMSE_train.append(RMSE_train)
    data_MAE_train.append(MAE_train)
    data_r2_test.append(r2_test)
    data_RMSE_test.append(RMSE_test)
    data_MAE_test.append(MAE_test)

    
    print('----------------------')
    print('seed:', seed)
    print('R2_train:', r2_train)
    print('R2_test:', r2_test)
    print('----------------------')
print('R2_train_means:', sum(data_r2_train)/10)
print('R2_test_means:', sum(data_r2_test)/10)

----------------------
seed: 0
R2_train: 0.8516586086061235
R2_test: 0.68209574332897
----------------------
----------------------
seed: 1
R2_train: 0.7811048868954963
R2_test: 0.8339554367790554
----------------------
----------------------
seed: 2
R2_train: 0.8408669011527474
R2_test: 0.7803532884375399
----------------------
----------------------
seed: 3
R2_train: 0.8299038376621485
R2_test: 0.7974578503206218
----------------------
----------------------
seed: 4
R2_train: 0.8224981701956882
R2_test: 0.75299572480365
----------------------
----------------------
seed: 5
R2_train: 0.8266746090812594
R2_test: 0.6562065841988205
----------------------
----------------------
seed: 6
R2_train: 0.8575982093806978
R2_test: 0.6994781216489139
----------------------
----------------------
seed: 7
R2_train: 0.8271149802239252
R2_test: 0.7430977012490396
----------------------
----------------------
seed: 8
R2_train: 0.8488810039837043
R2_test: 0.7177944562183373
----------------------
-----

In [5]:
data_r2_train_pd = pd.DataFrame(data= data_r2_train, columns=['r2_train'])
data_RMSE_train_pd = pd.DataFrame(data=data_RMSE_train, columns=['RMSE_train'])
data_MAE_train_pd = pd.DataFrame(data=data_MAE_train, columns=['MAE_train'])
data_r2_test_pd = pd.DataFrame(data=data_r2_test, columns=['r2_test'])
data_RMSE_test_pd = pd.DataFrame(data=data_RMSE_test, columns=['RMSE_test'])
data_MAE_test_pd = pd.DataFrame(data=data_MAE_test, columns=['MAE_test'])

data_all = pd.concat([data_r2_train_pd, data_RMSE_train_pd, data_MAE_train_pd, data_r2_test_pd, 
                      data_RMSE_test_pd, data_MAE_test_pd],
                     axis=1, join='inner')

data_all.loc['mean'] = data_all.mean()
data_all.loc['std'] = data_all.std(ddof=0)
print(data_all)

data_all.to_csv('../../score/DFT/Lasso_score.csv')

      r2_train  RMSE_train  MAE_train   r2_test  RMSE_test   MAE_test
0     0.851659   10.457869   6.121268  0.682096  20.391246  13.239076
1     0.781105   14.069434   8.795794  0.833955  12.683531   9.009283
2     0.840867   12.133173   7.893015  0.780353  14.574355   9.660633
3     0.829904   12.595655   7.890240  0.797458  13.942346   9.132972
4     0.822498   12.370278   7.761127  0.752996  16.206346   9.629299
5     0.826675   11.846865   6.977362  0.656207  20.448055  12.975461
6     0.857598   10.810430   6.809105  0.699478  19.017522  12.740002
7     0.827115   12.859400   8.004430  0.743098  15.260536  10.875549
8     0.848881   11.532665   7.101591  0.717794  16.699869  10.457056
9     0.791621   14.532056   9.926107  0.889938   8.806579   7.811174
mean  0.827792   12.320782   7.728004  0.755337  15.803039  10.553051
std   0.022555    1.165729   0.976297  0.065069   3.273525   1.693470
